# processor

> Fill in a module description here

In [ ]:
#| default_exp processor

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from typing import List, Dict, Any, Optional
from datetime import datetime
from llm_data_extractor.models import ExtractionResult

In [ ]:
#| export

def format_for_db(results: List[ExtractionResult], 
                  source_id: Optional[str] = None,
                  batch_id: Optional[str] = None) -> List[Dict[str, Any]]:
    """
    Format extraction results for database insertion.
    
    Args:
        results: List of ExtractionResult objects
        source_id: Optional identifier for the source document/text
        batch_id: Optional identifier for the processing batch
        
    Returns:
        List of dictionaries ready for database insertion
    """
    
    formatted_results = []
    
    for result in results:
        record = {
            'question_id': result.question_id,
            'raw_answer': result.raw_answer,
            'parsed_answer': _serialize_answer(result.parsed_answer),
            'confidence': result.confidence,
            'is_valid': result.is_valid,
            'validation_error': result.validation_error,
            'processed_at': result.timestamp.isoformat(),
            'source_id': source_id,
            'batch_id': batch_id
        }
        
        formatted_results.append(record)
    
    return formatted_results


def format_for_target_tables(results: List[ExtractionResult], 
                           questions_map: Dict[str, Any]) -> Dict[str, List[Dict[str, Any]]]:
    """
    Format results grouped by target table for direct insertion into business tables.
    
    Args:
        results: List of ExtractionResult objects
        questions_map: Map of question_id to Question objects
        
    Returns:
        Dictionary with table names as keys and records as values
    """
    
    table_records = {}
    
    for result in results:
        if not result.is_valid:
            continue  # Skip invalid results
            
        question = questions_map.get(result.question_id)
        if not question:
            continue
            
        table_name = question.target_table
        field_name = question.target_field
        
        if table_name not in table_records:
            table_records[table_name] = {}
        
        # Use source_id or batch_id as the record key
        # This assumes one record per source document per table
        record_key = 'default'  # You might want to pass this in
        
        if record_key not in table_records[table_name]:
            table_records[table_name][record_key] = {}
        
        table_records[table_name][record_key][field_name] = result.parsed_answer
        table_records[table_name][record_key][f'{field_name}_confidence'] = result.confidence
        table_records[table_name][record_key][f'{field_name}_processed_at'] = result.timestamp.isoformat()
    
    # Convert to list format for each table
    formatted_tables = {}
    for table_name, records in table_records.items():
        formatted_tables[table_name] = list(records.values())
    
    return formatted_tables


def create_summary_stats(results: List[ExtractionResult]) -> Dict[str, Any]:
    """
    Create summary statistics for a batch of extraction results.
    
    Args:
        results: List of ExtractionResult objects
        
    Returns:
        Dictionary with summary statistics
    """
    
    if not results:
        return {
            'total_questions': 0,
            'valid_answers': 0,
            'invalid_answers': 0,
            'success_rate': 0.0,
            'avg_confidence': 0.0
        }
    
    total_questions = len(results)
    valid_answers = sum(1 for r in results if r.is_valid)
    invalid_answers = total_questions - valid_answers
    success_rate = valid_answers / total_questions
    
    # Calculate average confidence for valid answers
    valid_confidences = [r.confidence for r in results if r.is_valid and r.confidence is not None]
    avg_confidence = sum(valid_confidences) / len(valid_confidences) if valid_confidences else 0.0
    
    # Get validation error categories
    error_types = {}
    for result in results:
        if not result.is_valid and result.validation_error:
            error_type = result.validation_error.split(':')[0]  # Get first part of error
            error_types[error_type] = error_types.get(error_type, 0) + 1
    
    return {
        'total_questions': total_questions,
        'valid_answers': valid_answers,
        'invalid_answers': invalid_answers,
        'success_rate': round(success_rate, 3),
        'avg_confidence': round(avg_confidence, 3),
        'error_types': error_types,
        'processed_at': datetime.now().isoformat()
    }


def _serialize_answer(answer: Any) -> str:
    """
    Serialize parsed answer for database storage.
    
    Args:
        answer: The parsed answer value
        
    Returns:
        String representation suitable for database storage
    """
    
    if answer is None:
        return None
    
    # Handle datetime objects
    if isinstance(answer, datetime):
        return answer.isoformat()
    
    # Handle lists and complex objects
    if isinstance(answer, (list, dict)):
        return json.dumps(answer)
    
    # Handle primitive types
    return str(answer)


def create_audit_record(batch_id: str, 
                       source_id: str,
                       total_questions: int,
                       processing_time_seconds: float,
                       llm_config: Dict[str, Any],
                       summary_stats: Dict[str, Any]) -> Dict[str, Any]:
    """
    Create an audit record for the extraction batch.
    
    Args:
        batch_id: Unique identifier for the batch
        source_id: Identifier for the source document
        total_questions: Number of questions processed
        processing_time_seconds: Time taken for processing
        llm_config: Configuration used for LLM calls
        summary_stats: Summary statistics from create_summary_stats
        
    Returns:
        Audit record dictionary
    """
    
    return {
        'batch_id': batch_id,
        'source_id': source_id,
        'total_questions': total_questions,
        'processing_time_seconds': round(processing_time_seconds, 2),
        'llm_model': llm_config.get('model_name'),
        'llm_temperature': llm_config.get('temperature'),
        'success_rate': summary_stats.get('success_rate'),
        'avg_confidence': summary_stats.get('avg_confidence'),
        'processed_at': datetime.now().isoformat(),
        'config_snapshot': json.dumps(llm_config)
    }

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()